In [1]:
import torch
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print(device)
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

mps:0
PyTorch version:2.1.1
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-14.1.1-arm64-arm-64bit


In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets,models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as lr_scheduler
import time
import copy
import numpy as np 
import pandas as pd 

In [3]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
dataset_train=datasets.ImageFolder("/Users/chosh/Downloads/PokemonData",transform=transform
                )
dataset_length=len(dataset_train)
trainset,valset=random_split(dataset_train,[int(0.8*dataset_length),dataset_length-int(0.8*dataset_length)])
print(len(dataset_train.classes))
loadeddata=DataLoader(dataset_train,batch_size=64,shuffle=True,num_workers=2)
loaded_train=DataLoader(trainset,batch_size=64,shuffle=True,num_workers=2)
loaded_val=DataLoader(valset,batch_size=64,shuffle=True,num_workers=2)
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print(device)
my_model=models.resnet50(pretrained=True)
print(my_model)

150
mps:0


/Users/chosh/anaconda3/envs/python39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/chosh/anaconda3/envs/python39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/chosh/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Ser

In [4]:
my_model.fc=nn.Linear(2048,150)
my_model=my_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(my_model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in [loaded_train,loaded_val][phase=='val']:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                inputs.unsqueeze(0)
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / [int(0.8*dataset_length),dataset_length-int(0.8*dataset_length)][phase=='val']
            epoch_acc = running_corrects.float() / [int(0.8*dataset_length),dataset_length-int(0.8*dataset_length)][phase=='val']

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(my_model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.8650 Acc: 0.0574
val Loss: 4.5768 Acc: 0.1723

Epoch 1/24
----------
train Loss: 4.1304 Acc: 0.3506
val Loss: 3.7386 Acc: 0.4443

Epoch 2/24
----------
train Loss: 3.1528 Acc: 0.6083
val Loss: 2.8037 Acc: 0.6085

Epoch 3/24
----------
train Loss: 2.2061 Acc: 0.7586
val Loss: 1.9682 Acc: 0.7617

Epoch 4/24
----------
train Loss: 1.5028 Acc: 0.8580
val Loss: 1.4150 Acc: 0.8277

Epoch 5/24
----------
train Loss: 1.0255 Acc: 0.9091
val Loss: 1.0467 Acc: 0.8702

Epoch 6/24
----------
train Loss: 0.7353 Acc: 0.9379
val Loss: 0.8173 Acc: 0.8871

Epoch 7/24
----------
train Loss: 0.5833 Acc: 0.9566
val Loss: 0.7989 Acc: 0.8878

Epoch 8/24
----------
train Loss: 0.5666 Acc: 0.9573
val Loss: 0.7862 Acc: 0.8915

Epoch 9/24
----------
train Loss: 0.5529 Acc: 0.9628
val Loss: 0.7737 Acc: 0.8878

Epoch 10/24
----------
train Loss: 0.5341 Acc: 0.9632
val Loss: 0.7453 Acc: 0.8944

Epoch 11/24
----------
train Loss: 0.5228 Acc: 0.9632
val Loss: 0.7374 Acc: 0.8915

Ep